# Homework 11 - Voice of Customer Analytics

## Prerequisite

### Install Library

In [1]:
!pip -q install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0
!pip -q install transformers==3.5.0 thai2transformers==0.1.2
!pip install --upgrade pythainlp
!pip install tensorflow_text

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.5.0 requires tensorflow<2.6,>=2.5.0, but you have tensorflow 2.3.3 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
  Using cached tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl (4.0 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: typing-extensions
    Found e

### Import Library

In [2]:
# Data imputation
import pandas as pd
import numpy as np

import re
import emoji

import torch

#datasets
from datasets import load_dataset

#transformers
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    pipeline,
)

#thai2transformers
import thai2transformers
from thai2transformers.preprocess import process_transformers
from thai2transformers.metrics import (
    classification_metrics, 
    multilabel_classification_metrics,
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

from tqdm.auto import tqdm

import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

import pythainlp
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from sklearn.metrics import classification_report

### Clean Text

In [3]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def cleaner(text):
    stopwords = list(pythainlp.corpus.thai_stopwords())
    removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ']
    screening_words = stopwords + removed_words
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = re.sub(r"\n", "", text) #Remove http links
    text = give_emoji_free_text(text)
    text = " ".join(text.split())
    text = pythainlp.word_tokenize(str(text), engine='newmm')
    full_text = ''
    for word in text:
      if word not in screening_words:
        full_text = full_text + word
    return full_text

## Import Data

In [4]:
df = pd.read_csv('/content/drive/MyDrive/BADS7105/CustomerReviews.csv')

In [5]:
df['Review_clean'] = df['Review'].apply(lambda x: give_emoji_free_text(x)).apply(lambda x: cleaner(x))

In [6]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_clean
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,บริการดีมากพนักงานน่ารักสะอาดสะอ้านใส่ใจประทับ...
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,นึกถึงชาบูญี่ปุ่นคิดถึงโมโม่พาราไดซ์คุณภาพเค้า...
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ทานนั่งโต๊ะหม้อโต๊ะยังมีฉากกั้นน้ำซุปเลือก2รสน...
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,นึกถึงชาบูเนื้อราคาโหดเกินไปนึกถึงโมโม่โปร4จ่า...
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,เดินหน้าได้กลิ่นชาบูหอมมากโปรบัตรเครดิตตัดสินใ...


## Analytics Model

### WangchanBERTa (Prepare)

In [7]:
model_name = "wangchanberta-base-att-spm-uncased"

#create tokenizer
tokenizer = AutoTokenizer.from_pretrained(
                f'airesearch/{model_name}',
                revision='main',
                model_max_length=416,)

### Sentiment Analysis (Pre-trained from WangchanBERTa)

In [8]:
dataset_name = "wongnai_reviews"

classify_multiclass = pipeline(task='sentiment-analysis',
         tokenizer=tokenizer,
         model = f'airesearch/{model_name}',
         revision = f'finetuned@{dataset_name}')

In [9]:
df['sentiment_class'] = df['Review_clean'].apply(process_transformers).apply(classify_multiclass)

In [10]:
df = df.explode('sentiment_class')
df.reset_index(drop = True, inplace = True)
df_2 = df['sentiment_class']
df = df.join(pd.json_normalize(df_2))

In [11]:
df.rename(columns={"label":"sentiment_label","score":"sentiment_score"}, inplace = True)

In [12]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_clean,sentiment_class,sentiment_label,sentiment_score
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,บริการดีมากพนักงานน่ารักสะอาดสะอ้านใส่ใจประทับ...,"{'label': '4', 'score': 0.5268824696540833}",4,0.526882
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,นึกถึงชาบูญี่ปุ่นคิดถึงโมโม่พาราไดซ์คุณภาพเค้า...,"{'label': '3', 'score': 0.5530749559402466}",3,0.553075
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ทานนั่งโต๊ะหม้อโต๊ะยังมีฉากกั้นน้ำซุปเลือก2รสน...,"{'label': '4', 'score': 0.7228193283081055}",4,0.722819
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,นึกถึงชาบูเนื้อราคาโหดเกินไปนึกถึงโมโม่โปร4จ่า...,"{'label': '4', 'score': 0.5433670878410339}",4,0.543367
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,เดินหน้าได้กลิ่นชาบูหอมมากโปรบัตรเครดิตตัดสินใ...,"{'label': '4', 'score': 0.837055504322052}",4,0.837056


In [13]:
print(classification_report(df[df['Rating'].notna()]['Rating'].astype(int), 
                            df[df['Rating'].notna()]['sentiment_label'].astype(int)))

              precision    recall  f1-score   support

           3       0.50      0.60      0.55         5
           4       0.62      0.73      0.67        11
           5       0.00      0.00      0.00         4

    accuracy                           0.55        20
   macro avg       0.37      0.44      0.40        20
weighted avg       0.46      0.55      0.50        20



### Topic Modeling (USE)

In [14]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [15]:
# Encode / Embed df_test

emb_review_full = embed(df['Review_clean'].values).numpy()
df_emb_review_full = pd.DataFrame(
                                    emb_review_full,
                                    index = df['Review_clean'].values
                                  )
df_emb_review_full.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
บริการดีมากพนักงานน่ารักสะอาดสะอ้านใส่ใจประทับใจบริการรสชาติอาหารอันนี้แปลก​เซ็ท560บาทอาหารจำกัด​​หลากหลาย​​พอกับความต้องการกินเนื้อ​กินสันคอ4ถาด​เป็ด1แฟนกินเนื้อ​​ภาพ​ดีน้ำซุป​ดำ​เผ็ดน้ำจิ้มอร่อยกินรสชาติแบบนี้เต้าเจี้ยวเคลมรสชาติดีถามราคาโดยรวมคุ้มโมจิอร่อยไอติมอร่อยไม่ต้องกินข้างนอกสรุปประทับใจบริการคุณภาพอาหาร,0.009749,-0.007666,-0.021905,-0.058955,0.029902,0.065597,0.063624,0.042563,-0.012610,-0.047974,0.059316,0.056420,-0.021398,-0.040703,0.033958,0.051908,-0.019666,-0.032234,-0.003156,0.030982,0.061772,-0.038180,0.009318,0.022722,-0.036228,-0.040628,-0.055533,-0.005264,0.021025,0.002186,-0.025991,0.072817,-0.001500,-0.057114,-0.079956,-0.010429,-0.011896,-0.001739,-0.015367,-0.043272,...,0.012615,-0.024269,-0.003723,-0.013496,0.036882,-0.057075,0.031154,0.050413,-0.014659,0.082144,-0.042401,0.042603,0.055203,0.069204,0.069986,-0.063284,-0.009466,-0.064005,-0.018685,0.022935,0.053350,-0.083610,-0.010286,0.017446,-0.053497,-0.006303,-0.068034,0.001177,-0.035568,0.012715,0.020036,0.030384,-0.065958,-0.074814,-0.022668,-0.058338,0.055598,-0.000618,0.076074,0.001482
นึกถึงชาบูญี่ปุ่นคิดถึงโมโม่พาราไดซ์คุณภาพเค้ามาตราฐานมีโอกาสมาทาน30นาทีบริการดีอาหารเครื่องดื่มที่ทางทะยอยแจ้งรายละเอียดการงดออร์เดอร์สาขาพนักงานบริการดีน่ารักเป็นกันเองเหวี่ยงเจอสาขาพระราม9โอเคพนักงานทำงานCTWพรีเมียมบริการสู้สาขาTerminal21ลูกค้า,-0.047744,0.063694,0.052632,-0.035745,0.058667,0.060377,0.001722,0.082430,-0.006086,0.062064,0.027727,-0.020115,0.011273,0.012659,0.059723,0.069512,0.023222,0.014943,-0.052432,0.034096,0.015083,0.008708,0.041945,-0.010237,-0.020473,0.031915,-0.042081,0.007543,0.063599,-0.068036,-0.023977,0.071950,0.055102,-0.015441,-0.026018,0.003448,-0.007243,-0.055297,0.038449,-0.011656,...,0.034346,0.051779,-0.001162,0.015425,0.048842,0.024703,0.065123,0.022600,-0.015681,0.081307,0.072698,-0.007978,-0.011979,0.056424,-0.062447,-0.066339,-0.053957,-0.028822,-0.063656,0.069119,0.076079,0.064761,0.029902,-0.002534,-0.039837,0.033434,-0.033268,0.012683,0.011245,0.078451,0.002411,0.042029,-0.055367,-0.022144,0.062088,-0.052038,-0.038451,0.004655,0.027382,-0.040541
"ทานนั่งโต๊ะหม้อโต๊ะยังมีฉากกั้นน้ำซุปเลือก2รสน้ำจิ้มแตกต่างกันตามน้ำซุปเลือกชาบูน้ำดำน้ำซุปเห็ดชาบูน้ำดำ:หวานเค็มรสชาติเข้มข้นเสิร์ฟไข่สดเป็นผู้ใหญ่ชอบหวานชอบน้ำซุปเห็ด:มีชื่อภาษาญี่ปุ่น555เสิร์ฟเห็ดน้ำจิ้มซอยซอสรสชาติจืดแนวน้ำซุปใสเนื้อสัตว์เลือกเนื้อวัว2,เนื้อหมู2,เนื้อเป็ด,ไก่ไม้ไผ่สั่งลองเนื้อวัว:อร่อยกกชิ้นเนื้อนุ่มกินชาบูน้ำดำเข้ากันเนื้อหมู:คุโรบูตะอร่อยนุ่มลิ้นเนื้อเป็ด:หนึบเหนียวคาวทานจานรู้สึกไก่ไม้ไผ่:ไก่บดลืมถ่ายรูปรสชาติกรุบแตาส่วนตัวโควิดบาร์ผักน้ำพนักงานตักอยากได้จิ้มเลือกของหวานไอติมไดฟูกุลองสั่งไดฟูกุไอติมไหวแย้วววสั่งสามรสสตอเบอรี่ชาเขียวถั่วแดงอร่อย::สรุป::อร่อยคุ้มค่าราคาทานตกคนละ600++ทานเนื้อวัวคุ้ม",0.008486,0.000147,0.018652,-0.016684,0.062367,0.064350,0.046910,0.050956,0.043066,-0.023219,0.038271,0.019959,-0.054710,-0.025447,0.063809,0.061813,0.014716,0.008518,0.052356,0.049809,0.055194,0.013711,-0.022107,-0.026036,-0.063434,-0.050575,-0.064646,0.017757,0.042076,-0.021358,-0.057066,0.045040,-0.053927,-0.045964,-0.063261,0.004655,-0.044631,-0.062235,-0.033863,-0.064733,...,-0.001747,-0.038633,-0.023822,0.001172,0.016270,-0.027581,-0.041086,0.056289,0.029640,0.060894,-0.058875,0.063422,0.061065,0.061357,0.060466,-0.060920,-0.028344,-0.034694,-0.047029,-0.036369,0.054422,-0.061938,0.026047,-0.064938,-0.011384,-0.049557,-0.026692,-0.001047,0.046790,0.056506,0.062581,-0.009032,-0.062090,-0.041980,0.005217,-0.013696,-0.003577,-0.022613,0.064414,-0.051938
นึกถึงชาบูเนื้อราคาโหดเกินไปนึกถึงโมโม่โปร4จ่าย3ธรรมดาส่วนตัวชอบเนื้อออสเกินไปอร่อยทานเลี่ยนไวชอบไอศกรีมรสชาเขียวรสข้าวสั่ง2ถ้วย,-0.036381,-0.014464,-0.042939,0.030470,-0.057348,0.066190,-0.056889,0.043056,-0.011354,0.016801,0.033810,0.0061

In [16]:
# define selected category
set_category = ['รสชาติ', 'บริการ', 'ความสะอาด', 'ราคา', 'สถานที่']

# create df which contains embedded vector of selected category
df_emb_category = pd.DataFrame(
                                embed(set_category).numpy(), 
                                index = set_category
                               )
df_emb_category.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
รสชาติ,0.111921,0.000967,0.050725,-0.066480,-0.040835,0.090405,-0.040258,0.040384,-0.031097,0.004043,0.040932,-0.014144,-0.042109,-0.002929,-0.038849,-0.023982,-0.030283,-0.032477,0.087952,0.048948,-0.018469,0.095193,-0.001878,0.026893,-0.027570,0.070072,-0.027992,0.004430,0.022624,0.030513,0.003889,0.051365,0.079605,-0.051678,-0.046669,-0.037999,0.068026,0.033628,-0.022359,-0.039875,...,0.061587,0.016347,-0.026035,-0.004947,0.030151,-0.000081,0.054121,0.052898,-0.043165,0.101000,-0.000239,0.043090,0.057274,-0.069179,0.002418,-0.068554,0.048447,0.022743,-0.069460,0.009254,0.030531,0.016023,-0.030343,0.006829,-0.057635,-0.000114,0.045995,-0.045680,0.025057,0.001572,-0.062543,-0.029302,0.007168,0.047672,-0.035372,0.013185,0.024466,0.043359,-0.003370,-0.002125
บริการ,0.150424,-0.019815,0.005300,0.014720,0.002672,-0.005768,0.022027,0.001578,-0.019698,-0.044280,-0.024475,-0.070359,-0.081485,-0.050700,0.001885,-0.045457,-0.059570,0.072177,-0.034475,-0.048325,0.085761,0.058215,0.054968,-0.013415,-0.042003,0.044097,0.084193,-0.008577,-0.008273,-0.051483,-0.051302,0.094554,-0.023673,-0.023931,-0.067230,-0.052318,0.028369,0.081185,-0.005062,0.016900,...,0.052865,-0.084915,0.047328,0.066514,0.062678,0.050427,0.054779,-0.036502,0.074500,0.067458,0.070705,-0.002656,0.013970,-0.016277,0.037893,-0.065666,0.007628,-0.033731,0.032810,0.065954,0.045204,0.013109,0.010636,-0.005719,0.025102,0.033776,0.043276,0.089827,-0.050681,0.055838,-0.053757,-0.036840,-0.057552,-0.004938,0.008403,-0.028433,-0.017613,-0.003656,0.039623,-0.064631
ความสะอาด,0.119814,-0.019158,0.068316,0.011222,-0.070133,0.056906,-0.012893,-0.027539,0.012397,0.021955,0.058986,0.036164,-0.081184,-0.034089,-0.112333,0.013705,-0.006926,0.019979,0.004234,0.024327,0.098352,0.032904,0.032747,0.021494,-0.050842,0.021008,-0.043911,-0.052837,-0.023529,-0.065205,-0.040494,0.005594,-0.056694,-0.015889,-0.051742,-0.055823,0.078863,0.011285,-0.027791,0.062388,...,0.012639,0.013749,0.029831,-0.018968,0.016032,-0.042822,-0.028215,-0.036844,0.005552,0.076438,-0.008318,-0.001211,-0.016614,-0.049861,0.068970,-0.078520,-0.058477,0.049101,0.048673,0.060904,-0.059462,-0.042498,0.045134,0.042138,-0.036081,-0.006884,-0.017229,0.049526,-0.046065,-0.009041,-0.071292,-0.016704,0.002123,0.007636,0.013441,0.002337,0.030567,0.010901,-0.046789,-0.006601
ราคา,0.141932,-0.026150,0.029388,-0.009860,-0.048712,-0.001908,-0.073864,0.025503,-0.004929,0.009823,-0.010537,-0.051046,-0.003839,-0.011837,0.038502,0.055992,0.034541,0.006195,0.039493,0.126899,-0.012339,0.118112,0.066793,-0.006069,0.021026,0.014053,0.013093,0.066908,0.001283,-0.019515,-0.006068,-0.011540,-0.000885,0.059452,0.049519,-0.069880,0.012388,0.018428,-0.041971,-0.025006,...,0.024511,0.033336,-0.001110,-0.003090,0.004730,0.062371,-0.044670,0.001805,-0.028933,0.113941,0.044655,0.025965,0.001900,-0.045246,0.010670,-0.032343,0.064686,-0.002833,-0.017151,-0.009384,-0.007695,0.035895,-0.076200,0.031835,-0.052720,0.038725,0.100163,0.001695,-0.037687,-0.005750,-0.070597,0.016963,0.037373,0.008691,-0.020105,0.034799,0.078453,0.095272,0.013083,-0.039743
สถานที่,0.138405,0.008914,-0.004131,0.026843,0.038214,-0.021540,0.036218,0.007485,0.000502,-0.028377,-0.004969,-0.026508,-0.079137,-0.073594,0.005913,0.017371,-0.065814,0.008175,0.030834,-0.020924,-0.035333,0.064043,0.039515,0.022434,-0.020538,0.067393,-0.030589,0.013809,0.036839,0.003607,0.042527,0.046549,0.022660,-0.042437,-0.044491,-0.067624,-0.030648,0.009791,0.075406,-0.025341,...,-0.022716,-0.019213,0.071998,-0.029391,-0.074661,-0.021706,-0.006352,-0.004044,0.022321,0.076951,-0.029785,-0.013094,0.013554,0.021847,-0.035868,-0.075024,0.049061,-0.010427,0.042079,-0.030927,0.067264,0.048898,-0.034424,0.002974,0.054593,-0.015547,0.083314,-

In [17]:
argmax_cal = df_emb_review_full.dot(df_emb_category.T)#.idxmax('columns')#.values
argmax_category = argmax_cal.idxmax('columns').values
argmax_val = argmax_cal.max('columns').values
df_result = pd.DataFrame({
                        'original_text' : df_emb_review_full.index,
                        'topic_category' : argmax_category,
                        'topic_val' : argmax_val
                      })
argmax_cal.reset_index(inplace = True)

In [18]:
df = df.join(df_result[["topic_category","topic_val"]]).join(argmax_cal)

In [19]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_clean,sentiment_class,sentiment_label,sentiment_score,topic_category,topic_val,index,รสชาติ,บริการ,ความสะอาด,ราคา,สถานที่
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,บริการดีมากพนักงานน่ารักสะอาดสะอ้านใส่ใจประทับ...,"{'label': '4', 'score': 0.5268824696540833}",4,0.526882,รสชาติ,0.264941,บริการดีมากพนักงานน่ารักสะอาดสะอ้านใส่ใจประทับ...,0.264941,0.120919,0.092882,0.099882,-0.005179
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,นึกถึงชาบูญี่ปุ่นคิดถึงโมโม่พาราไดซ์คุณภาพเค้า...,"{'label': '3', 'score': 0.5530749559402466}",3,0.553075,บริการ,0.217780,นึกถึงชาบูญี่ปุ่นคิดถึงโมโม่พาราไดซ์คุณภาพเค้า...,0.118777,0.217780,0.011392,-0.029482,-0.036086
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ทานนั่งโต๊ะหม้อโต๊ะยังมีฉากกั้นน้ำซุปเลือก2รสน...,"{'label': '4', 'score': 0.7228193283081055}",4,0.722819,รสชาติ,0.238783,ทานนั่งโต๊ะหม้อโต๊ะยังมีฉากกั้นน้ำซุปเลือก2รสน...,0.238783,0.027202,0.083478,0.057245,0.067891
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,นึกถึงชาบูเนื้อราคาโหดเกินไปนึกถึงโมโม่โปร4จ่า...,"{'label': '4', 'score': 0.5433670878410339}",4,0.543367,รสชาติ,0.309807,นึกถึงชาบูเนื้อราคาโหดเกินไปนึกถึงโมโม่โปร4จ่า...,0.309807,-0.040354,0.063562,0.133724,0.012788
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,เดินหน้าได้กลิ่นชาบูหอมมากโปรบัตรเครดิตตัดสินใ...,"{'label': '4', 'score': 0.837055504322052}",4,0.837056,รสชาติ,0.173144,เดินหน้าได้กลิ่นชาบูหอมมากโปรบัตรเครดิตตัดสินใ...,0.173144,0.121809,0.047945,-0.023550,-0.001105
